Метрические методы классификации
1. Методы ближайших соседей
2. Настройка оптимального числа ближайших соседей в методе kNN
3. Выбор метрики в методе kNN
4. Другие метрические методы

# Методы ближайших соседей

*1. Подключитесь к одному из наборов данных на Kaggle: Вариант 3: Bank marketing.*

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Загрузка данных
df = pd.read_csv('/kaggle/input/bank-marketing-dataset/bank.csv')
df.head()

In [ ]:
df.keys()

In [ ]:
#df.info()

*2. Извлеките целевой признак (target). Какая из задач обучения с учителем рассматривается –– классификация или регрессия?*

Целевым признаком (target) является 'deposit', принимающий значения "yes" и "no", что означает положит клиент срочный депозит или нет. Рассматривается категория - обучение с учителем и тип задачи - классификация (предсказание категории объекта).


*3. Каково распределение значений target-переменной? Постройте подходящую визуализацию. Прокомментируйте результат.*

In [ ]:
import seaborn as sns
sns.catplot(x = "deposit", kind = "count", palette = "ch:.25", data = df)

In [ ]:
# Выведем процентное соотношение
df['deposit'].value_counts(normalize=True)

In [ ]:
# print(df['deposit'].shape) # Всего 11162 строк
df['deposit']

Имеем ~52,6% клиентов без депозитов и ~47,4% клиентов c депозитом.

Можно заметить, что классы 'no' и 'yes' почти сбалансированы.

*4. Проведите необходимую предобработку данных (preprocessing). Для построения моделей с помощью метрических методов все признаки должны быть закодированы числами. Полезными будут следующие методы библиотеки Pandas:*
1. map() – для перекодировки категориальной переменной числовыми метками; 
2. get_dummies() – для создания нескольких бинарных признаков на основе категориального. 

*Также может потребоваться масштабирование данных (scaling). Воспользуйтесь классом StandardScaler библиотеки Scikit-learn.*

In [ ]:
from sklearn. preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df.default)
df['default_le'] = le.transform(df.default)

# ручная альтернатива
#dct = {'no': 0, 'yes': 1} # словарь для кодировки 
#df['default_le'] = df['default'].map(dct)

le_1 = LabelEncoder()
le_1.fit(df.housing)
df['housing_le'] = le_1.transform(df.housing)

le_2 = LabelEncoder()
le_2.fit(df.housing)
df['loan_le'] = le_2.transform(df.loan)

le_3 = LabelEncoder()
le_3.fit(df.housing)
df['deposit_le'] = le_3.transform(df.deposit)

dct_1 = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec':12}
df['month_le'] = df['month'].map(dct_1)

#dct_2 = {'married': 1, 'single': 2, 'divorced': 3}
#df['marital_le'] = df['marital'].map(dct_2)

#dct_3 = {'secondary': 1, 'tertiary': 2, 'primary': 3, 'unknown': 4}
#df['education_le'] = df['education'].map(dct_3)

#dct_4 = {'unknown': 0, 'cellular': 1, 'telephone': 2}
#df['contact_le'] = df['contact'].map(dct_4)

#dct_5 = {'unknown': 0, 'other': 1, 'failure': 2, 'success': 3}
#df['poutcome_le'] = df['poutcome'].map(dct_5)

df.head().T

In [ ]:
df_1 = pd.read_csv('/kaggle/input/bank-marketing-dataset/bank.csv')
df_1['default'] = df['default_le']
df_1['housing'] = df['housing_le']
df_1['loan'] = df['loan_le']
df_1['deposit'] = df['deposit_le']
df_1['month'] = df['month_le']

#df_1['marital'] = df['marital_le']
#df_1['education'] = df['education_le']
#df_1['contact'] = df['contact_le']
#df_1['poutcome'] = df['poutcome_le']

# get_dummies
df_1 = pd.get_dummies(df_1, columns=['marital', 'education', 'contact', 'poutcome'])

df_1.head().T

In [ ]:
#df_1.info()

In [ ]:
#df_2 = df_1.drop(['job'], axis = 1)
df_2 = pd.get_dummies(df_1, columns=['job']) 

df_2.head().T

In [ ]:
#df_2.info()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Создание X - вся таблица без target (deposit), а y - target (deposit).

y = df_2['deposit']
df_3 = df_2.drop('deposit', axis = 1)

X = df_3

X_new = scaler.fit_transform(X)
X_new

*5. Разбейте набор данных на обучающую и валидационную (тестовую) выборки с помощью метода train_test_split.*

In [ ]:
# Разбиение
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, 
                                                      y, 
                                                      test_size=0.25, 
                                                      random_state=20) 

#random_state. Controls the shuffling applied to the data before applying the split. 

In [ ]:
print(X_train.shape, y_train.shape)
print( X_valid.shape, y_valid.shape)

*6. Обучите алгоритм классификации kNeighborsClassifier или регрессии KNeighborsRegressor . Оцените качество каждой модели на валидационной выборке с помощью*
1. accuracy_score для классификации;
2. mean_squared_error для регрессии.

*Сравните результаты и сделайте выводы.*

In [ ]:
#Обучение классификатора
# Создаём представителя класса модели, задаём необходимые гиперпараметры

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)

In [ ]:
# Обучаем модель на обучающей выборке
knn.fit(X_train, y_train)

In [ ]:
# Строим предсказания на основе обученной модели
y_pred = knn.predict(X_valid)
y_pred

In [ ]:
#Функция mean_squared_error вычисляет среднеквадратичную ошибку, метрику риска, 
#соответствующую ожидаемому значению квадратичной ошибки или убытка.

from sklearn.metrics import mean_squared_error
mean_squared_error(y_valid, y_pred)

In [ ]:
# Вычисляем метрику
knn.score(X_valid, y_valid)

In [ ]:
# Ещё один способ для вычисления метрики
from sklearn.metrics import accuracy_score
print(accuracy_score(y_valid, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_valid, y_pred))

Cреднеквадратическая ошибка мала, а score достаточно большое, откуда следует, что  модель можно считать качественной.

Также можно сделать вывод, что модель предсказывает случаи 'no' немного лучше чем 'yes', что очевидно, ведь ноборов 'no' в данном датасете больше. 

# Настройка оптимального числа ближайших соседей в методе kNN

*1. Создайте генератор разбиений, который перемешивает выборку перед созданием блоков ( shuffle=True ). Число блоков n_splits равно 5. Задайте также параметр random_state для воспроизводимости результатов. Например: kf = KFold(n_splits=5, shuffle=True, random_state=42).*

*Найдите показатель качества модели kNN на кросс-валидации. Подумайте, приемлемо ли использование вашей меры (метрики) качества в данной задаче? При необходимости пересчитайте качество с помощью другой метрики из списка.*

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

scores = cross_val_score(knn, X, y, cv = kf, scoring = 'accuracy')
scores.mean()

В лучщем случае метод должен вернуть положительное значение с плавающей запятой (лучшее - 1). Имеется число наближенное к 1-е, следовательное использованиие этой метрики для данной задачи 
приемлимо. 

Для примера,рассмотрим ещё F1 score. The F1 score is the harmonic mean of the precision and recall. The highest possible value of F1 is 1, indicating perfect precision and recall, and the lowest possible value is 0, if either the precision or the recall is zero.

In [ ]:
scores = cross_val_score(knn, X, y, cv = kf, scoring = 'f1')
scores.mean()

Полученное значение наближенно к единице,но хуже предыдущего случая.

*2. Осуществите кросс-валидацию модели при числе соседей k ∈ [1;50]. Используйте GridSearchCV . При каком k качество получилось наилучшим? Чему равна эта оценка качества? Постройте график значений метрики в зависимости от k (matplotlib.pyplot.plot()).*

In [ ]:
from sklearn.model_selection import GridSearchCV
knn_params = {'n_neighbors': np.arange(1, 51)}
knn_grid = GridSearchCV(knn, 
                        knn_params, 
                        scoring='f1',
                        cv = kf)
knn_grid.fit(X_train, y_train)

In [ ]:
print("Best_estimator: ", knn_grid.best_estimator_)
print("Cross-validated score of the best_estimator: ", knn_grid.best_score_)
print( "Best_index_ while the best_score_ attribute will not be available: ", knn_grid.best_params_)

In [ ]:
cv_results = pd.DataFrame(knn_grid.cv_results_)
cv_results.T

In [ ]:
# Предсказания на тестовой выборке для оптимального числа соседей
y_pred = knn_grid.predict(X_valid)
accuracy_score(y_valid, y_pred)

In [ ]:
best_knn = KNeighborsClassifier(n_neighbors=13)
y_pred = best_knn.fit(X_train, y_train).predict(X_valid)
accuracy_score(y_valid, y_pred)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(cv_results["param_n_neighbors"],cv_results["mean_test_score"])

plt.xlabel('Number of neighbors')
plt.ylabel('Test accuracy')
plt.title('F1 score')
plt.show()

 При k = 13 качество получилось наилучшим.


# Выбор метрики в методе kNN

*1. Переберите разные варианты значений параметра p по сетке от 1 до 10 с таким шагом, чтобы всего было протестировано 200 вариантов (удобно использовать функцию numpy.linspace ). Используйте KNeighborsClassifier или KNeighborsRegressor с оптимальным значением n_neighbors , найденным ранее. Задайте опцию weights='distance' – данный параметр добавляет в алгоритм веса, зависящие от расстояния до ближайших соседей. В качестве метрики качества снова используйте accuracy . Качество оценивайте с помощью кросс-валидации по 5 блокам.*

In [ ]:
knn_params = {"p": np.linspace(1,10, 200)}

knn = KNeighborsClassifier(n_neighbors = 13, weights = "distance", n_jobs = -1)
knn.fit(X_train, y_train)

cv = GridSearchCV(knn, knn_params, cv = kf, scoring="f1")
cv.fit(X_train, y_train)

*2. Определите, при каком p качество на кросс-валидации оказалось оптимальным. Обратите внимание, что cross_val_score возвращает массив показателей качества по блокам; необходимо максимизировать среднее этих показателей.*

In [ ]:

print("Лучшее значение:", cv.best_score_)
print(cv.best_estimator_)

In [ ]:
cv_result = pd.DataFrame(cv.cv_results_)
cv_result

In [ ]:
plt.plot(cv_result["param_p"],cv_result["mean_test_score"])

# Другие метрические методы

*Поэкспериментируйте с другими метрическими методами для задач регрессии и классификации, представленными в библиотеке Scikit-learn:*
1. RadiusNeighborsClassifier;
2. RadiusNeighborsRegressor;
3. NearestCentroid

In [ ]:
from sklearn.neighbors import NearestCentroid
clf = NearestCentroid()
clf.fit(X_train, y_train)
NearestCentroid()

y_pred_nc = clf.predict(X_valid)
y_pred_nc

In [ ]:
mean_squared_error(y_valid, y_pred_nc)

In [ ]:
print(accuracy_score(y_valid, y_pred_nc))

In [ ]:
print(confusion_matrix(y_valid, y_pred_nc))

Метод NearestCentroid в сравнении с kNN даёт хуже результат, так как среднеквадратическая ошибка по значению близка к score.